In [ ]:
"""
        돈       사랑         저는       좋아요
줄거리1: 0         1          1           1
줄거리2: 1         0          1           1
줄거리3: 2         0          2           2
"""

In [7]:
import numpy as np
from numpy.linalg import norm

In [4]:
mov1=np.array([0,1,1,1])
mov2=np.array([1,0,1,1])
mov3=np.array([2,0,2,2])

In [5]:
np.dot(mov1,mov2)

2

In [11]:
np.dot(mov1,mov2)/(norm(mov1)*norm(mov2))  # <-cos 유사도 값

0.6666666666666667

In [12]:
def cos_sim(X,Y):
    return np.dot(X,Y)/(norm(X)*norm(Y))

In [14]:
print(cos_sim(mov1,mov2))
print(cos_sim(mov2,mov3))
print(cos_sim(mov1,mov3))

0.6666666666666667
1.0000000000000002
0.6666666666666667


In [ ]:
"""ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ연습문제 시작ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ"""

In [24]:
import pandas as pd
data=pd.read_csv("movies_metadata.csv")
data=data.head(20000)  # <- 너무커서 2만개만 짤라서 해봄

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
data['overview'].isnull().sum()
data['overview']=data['overview'].fillna('')

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english') 
# <- english에 해당하는 stopword(불용어:의미없는단어)가 제거
tfidf_mat=tfidf.fit_transform(data['overview'])
tfidf_mat.shape

(20000, 47487)

In [22]:
tfidf1=TfidfVectorizer() 
# <- 제거 안한
tfidf_mat1=tfidf1.fit_transform(data['overview'])
tfidf_mat1.shape

(20000, 47788)

In [30]:
tfidf_mat

<1x47487 sparse matrix of type '<class 'numpy.float64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [ ]:
data['title']
#Toy story와 가장 코사인 유사도가 큰 영화 10편 출력
# -> 영화의 제목과 유사도 출력

In [28]:
import nltk # <- natural language tool kit
nltk.__version__

'3.4.5'

In [31]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_mat,tfidf_mat)

In [35]:
indices=pd.Series(data.index,index=data['title']).drop_duplicates()
indices.head() #data의 'title'에 index를 줘서 번호를 뽑기 위해

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [55]:
test=indices['Toy Story']
sim_scores=list(enumerate(cosine_sim[test]))
print(sim_scores[1])

(1, 0.01575747731678539)


In [44]:
# recommendation 함수 만들기
def get_recommendations(targetmovie,cosine_sim=cosine_sim):
    idx=indices[targetmovie] # <- 비교 영화의 번호를 뽑음
    sim_scores=list(enumerate(cosine_sim[idx])) # <- cos유사도를 구하고 유사도를 리스트로 만들기
    #유사도에 따라 sort하기
    sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
    sim_scores=sim_scores[1:11] #<-10번째 영화까지 선별
    movie_indices = [i[0] for i in sim_scores]
    return ("유사한영화10개:",data['title'].iloc[movie_indices],
            "각 영화 cos유사도:",sim_scores)
get_recommendations("Toy Story")

('유사한영화10개:',
 15348               Toy Story 3
 2997                Toy Story 2
 10301    The 40 Year Old Virgin
 8327                  The Champ
 1071      Rebel Without a Cause
 11399    For Your Consideration
 1932                  Condorman
 3057            Man on the Moon
 485                      Malice
 11606              Factory Girl
 Name: title, dtype: object,
 '각 영화 cos유사도:',
 [(15348, 0.5258229300737997),
  (2997, 0.462638821864108),
  (10301, 0.2795541150488133),
  (8327, 0.20068589066985557),
  (1071, 0.18272968722756538),
  (11399, 0.15689199223985253),
  (1932, 0.15273436907628185),
  (3057, 0.14319886672358997),
  (485, 0.1375106791269381),
  (11606, 0.133606616102742)])